## 函数

In [50]:
def no_op():
    # 空语句
    pass

### 参数

#### 参数类型

函数参数包括*位置参数*（Positional Argument）和*关键字参数*（Keyword Argument, Named Arguments）；

In [9]:
# 函数的 `__doc__` 属性保存函数的注释
print(print.__doc__)

print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)

Prints the values to a stream, or to sys.stdout by default.
Optional keyword arguments:
file:  a file-like object (stream); defaults to the current sys.stdout.
sep:   string inserted between values, default a space.
end:   string appended after the last value, default a newline.
flush: whether to forcibly flush the stream.


#### 可变参数

`*` 是可变参数的标识；
- `*` 将多个参数打包成 tuple，用于传递多个非关键字参数；
- `**` 将多个参数打包成字典，调用时参数必须成对出现并用等号区分键和值，用于传递多个关键字参数；

In [1]:
def sum(*args):
    print(type(args))
    sum = 0
    for idx in range(0, len(args)):
        sum += args[idx]
    return sum
print(sum(1, 2, 3, 4))

def foo(**kwargs):
    if len(kwargs) == 0:
        print("Empty")
    else:
        print(kwargs)
foo(a = 1, b = 2)

<class 'tuple'>
10
{'a': 1, 'b': 2}


#### 默认参数

`function.__defaults__` 可以查看默认参数的值；

In [20]:
# 默认参数务必 immutable
def append_end_bug(l = []):
    l.append('END')
    return l
append_end_bug()
append_end_bug()
# l 是引用类型，每次调用会被更新
print(append_end_bug()) # ['END', 'END', 'END']

def append_end_propoer(l = None):
    if l is None:
        l = []
    l.append('END')
    return l
append_end_propoer()
append_end_propoer()
print(append_end_propoer())

['END', 'END', 'END']
['END']


#### packing, unpacking

In [5]:
# 可迭代对象（除字典）
def sum_3(a, b, c): # unpacking
    print(a, b, c)
a_list = [1, 2, 3]
sum_3(*a_list)



# 字典类型
def bar(name, age): # unpacking
    print(name, age)
a_dict = {'name': 'alice', 'age': 11}
bar(**a_dict)

1 2 3
alice 11


#### 传引用

Python 中所有参数传递都是传引用；

In [31]:
def foo(x):
    print('inside function, before mutation\t', id(x))
    x += 1
    # 发生更新时实际上是对副本进行操作
    print('inside function, after mutation\t\t', id(x), 'different')
x = 1
print('outside function, before invoking\t', id(x))
foo(x)
print('outside function, after invoking\t', id(x))

outside function, before invoking	 140525695254768
inside function, before mutation	 140525695254768
inside function, after mutation		 140525695254800 different
outside function, after invoking	 140525695254768


### 返回值

Python 函数只能返回一个值，由于存在语法糖而看上去可以返回多个值；

## 高阶函数

函数也是对象，可以作为函数参数和返回值（函数式编程的特征）；

### `lambda`

In [34]:
# 匿名函数；只能有一个表达式，无需 return 语句，表达式的结果就是返回的结果
lambda x, y: x + y;

### `filter`

保留返回 `True` 的元素；

In [40]:
a_list = [2, 18, 9, 22]
print(list(filter(lambda x: x % 3 == 0, a_list)))
filter?

[18, 9]


Init signature: filter(self, /, *args, **kwargs)
Docstring:     
filter(function or None, iterable) --> filter object

Return an iterator yielding those items of iterable for which function(item)
is true. If function is None, return the items that are true.
Type:           type
Subclasses:     

### `map`

In [42]:
sum = map(lambda a, b: a + b, [1, 2, 3], [4, 5, 6])
print(list(sum))

[5, 7, 9]


### `reduce`

In [49]:
from functools import reduce

a_list = [1, 2, 3, 4]
print('sum:', reduce(lambda x, y: x + y, a_list))
print('max:', reduce(lambda x, y: x if x > y else y, a_list))

sum: 10
max: 4


## Generator

生成器是一个生成元素的函数；

### Generator Expression

In [55]:
n = 10
## generator expression
g = (x**2 for x in range(n) if x % 2 == 0)
print(g, "\ntype:", type(g))

print(next(g))
print(g.__next__())
for other in g:
    print(other)

<generator object <genexpr> at 0x7fce8eb46d50> 
type: <class 'generator'>
0
4
16
36
64


### `yield`

In [66]:
def fibonacci(t):
    n, a, b = 0, 0, 1
    while n < t:
        yield b # do generate
        a, b = b, a + b
        n += 1
    return 'done'
fib_gen = fibonacci(10)
for other in fib_gen:
    print(other, end=' ')
print(next(fib_gen))

1 1 2 3 5 8 13 21 34 55 

StopIteration: 

### 执行过程

- 每次调用 `next()` 时开始执行，遇到 `yield` 就暂停，下次从暂停处继续执行；
- generator 中 `return` 的值只能在 `StopIteration` 时通过 `StopIteration.value` 获得；

## Iterator

迭代器可以理解是访问元素的指针；

所有迭代器都实现了 `__iter__()` 和 `__next__()` 方法；
- `__iter__()` 返回迭代器对象；
- `__next__()` 返回下一个元素值；

In [68]:
a_list = [1, 2, 3]
a_it = iter(a_list)
while True:
    try:
        print(next(a_it))
    except StopIteration:
        print('out of bounds')
        break

1
2
3
out of bounds


In [84]:
from itertools import islice

class Fibonacci:
    def __init__(self):
        self.previous, self.current = 0, 1
    
    def __iter__(self):
        return self

    def __next__(self):
        value = self.current
        self.previous, self.current = self.current, self.current + self.previous
        return value

fib = Fibonacci()
fib_iter = islice(fib, 0, 10)
# print(list(fib_iter))
for ele in fib_iter:
    print(ele, end=' ')
    
islice?

1 1 2 3 5 8 13 21 34 55 

Init signature: islice(self, /, *args, **kwargs)
Docstring:     
islice(iterable, stop) --> islice object
islice(iterable, start, stop[, step]) --> islice object

Return an iterator whose next() method returns selected values from an
iterable.  If start is specified, will skip all preceding elements;
otherwise, start defaults to zero.  Step defaults to one.  If
specified as another value, step determines how many values are
skipped between successive calls.  Works like a slice() on a list
but returns an iterator.
Type:           type
Subclasses:     